Next word prediction:
google and many other platforms use this-> automatically suggest list of words.

how is this possible?
searches human made and these searches are saved as data in cache file, using this data it suggest next word.

we will use 3 words and make prediction on that.

we will use LSTM:
long short-term memory
LSTM is a well-known and widely used idea in the development of recurrent neural networks (RNN).
    

# Importing Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os


# Load and Pre-process the data

In [2]:
file = open("pride and prejudice.txt","r",encoding = "utf8")

#store file in list
lines = []
for i in file:
    lines.append(i)

#convert list to string
data = ""
for i in lines:
    data = ' '.join(lines)

#replace unnecessary stuff with space like = new line,carriage return, unicode
data = data.replace('/n','').replace('\r','').replace('\ufeff','').replace('“','').replace('”','')

#remove unnecessary spaces
data = data.split()
data = ' '.join(data)
data[:500]

'The Project Gutenberg eBook of Pride and Prejudice, by Jane Austen This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using th'

In [3]:
len(data)

698418

# Apply Tokenization

In [4]:
# Tokenization is breaking the raw text into small chunks.
# tokenization breaks the raw text into words, sentences called tokens. 
# These tokens help in understanding the context or developing the model for the NLP.
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

#saving the tokenized file using pickle
pickle.dump(tokenizer,open("next_word_predict.pkl",'wb'))

#to convert text to number(sequence)
sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[1, 176, 158, 916, 3, 321, 4, 1171, 30, 72, 2534, 41, 916, 23, 21]

In [5]:
len(sequence_data)

125309

In [6]:
# unique words in file
vocab_size = len(tokenizer.word_index)+1
print(vocab_size)

7030


In [7]:
# inserting 4 words ion list out of which 3 words are used for prediction and 4th word is output
# i = 4
#i-3 = 4-3 =1
#i+1 = 4+1 = 5

sequence = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequence.append(words)
    
print("The Length of sequence are: ",len(sequence))
sequence = np.array(sequence)
sequence[:10]

The Length of sequence are:  125306


array([[   1,  176,  158,  916],
       [ 176,  158,  916,    3],
       [ 158,  916,    3,  321],
       [ 916,    3,  321,    4],
       [   3,  321,    4, 1171],
       [ 321,    4, 1171,   30],
       [   4, 1171,   30,   72],
       [1171,   30,   72, 2534],
       [  30,   72, 2534,   41],
       [  72, 2534,   41,  916]])

In [8]:
x = []
y = []

for i in sequence:
    x.append(i[0:3])
    y.append(i[3])
    
x = np.array(x)
y = np.array(y)

In [9]:
print("Data: ",x[:10])
print("Response: ",y[:10])

Data:  [[   1  176  158]
 [ 176  158  916]
 [ 158  916    3]
 [ 916    3  321]
 [   3  321    4]
 [ 321    4 1171]
 [   4 1171   30]
 [1171   30   72]
 [  30   72 2534]
 [  72 2534   41]]
Response:  [ 916    3  321    4 1171   30   72 2534   41  916]


In [10]:
y = to_categorical(y, num_classes = vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# Creating Model

In [11]:
model = Sequential()
model.add(Embedding(vocab_size,10,input_length = 3))
model.add(LSTM(1000,return_sequences= True))
model.add(LSTM(1000))
model.add(Dense(1000,activation="relu"))
model.add(Dense(vocab_size,activation = "softmax"))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             70300     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 7030)              7037030   
                                                                 
Total params: 20,156,330
Trainable params: 20,156,330
Non-trainable params: 0
_________________________________________________________________


# Plot Model

In [13]:
#from tensorflow import keras
#from keras.utils.vis_utils import plot_model

#keras.utils.plot_model(model,to_file = "plot.png",show_layer_names = True)

# Build Model

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint

#checkpoint = ModelCheckpoint("next_words.h5",monitor = "loss",verbose = 1,save_best_only = True)
#model.compile(loss = "categorical_crossentropy",optimizer = Adam(learning_rate = 0.001))
#model.fit(x,y, epochs =70, batch_size = 64,callbacks = [checkpoint])

# Prediction

In [15]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

#load the model and tokenizer

model = load_model('next_words.h5')
tokenizer = pickle.load(open("next_word_predict.pkl","rb"))

In [16]:
def Predict_Next_Words(model,tokenizer,text):
    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    preds = np.argmax(model.predict(sequence))
    predicted_word = ""
    
    for key, value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break
        
    print(predicted_word)
    return predicted_word

In [ ]:
while(True):
    text = input("Enter your line: ")
    
    if text == "0":
        print("Execution completed....")
        break
    
    else:
        try:
            text = text.split(" ")
            text = text[-3:]
            print(text)
            
            Predict_Next_Words(model,tokenizer,text)
            
        except Exception as e:
            print("Error Occurred: ",e)
            continue

Enter your line: he wants to
['he', 'wants', 'to']
1/1 [==============================] - 1s 1s/step
be
Enter your line: wealthy matches to
['wealthy', 'matches', 'to']
1/1 [==============================] - 1s 927ms/step
say
Enter your line: woman are saying
['woman', 'are', 'saying']
1/1 [==============================] - 0s 30ms/step
and
Enter your line: Mrs. Bennet deigned
['Mrs.', 'Bennet', 'deigned']
1/1 [==============================] - 0s 47ms/step
to
